In [42]:
install.packages("VancouvR")
install.packages("geojsonio")

library(geojsonio)
library(tidyverse)
library(dplyr)
library("knitr")

Warning message:
"package 'VancouvR' is in use and will not be installed"
Warning message:
"package 'geojsonio' is in use and will not be installed"


In [43]:
install.packages("cancensus")
library(cancensus)
options(cancensus.api_key='CensusMapper_f228791d9506a7a747ece66db73b95be')
options(cancensus.cache_path = "/cache")

Warning message:
"package 'cancensus' is in use and will not be installed"


In [44]:
census_data <- get_census(dataset='CA16', regions=list(CT="9330044.00"), vectors=c("v_CA16_401","v_CA16_404","v_CA16_1","v_CA16_2426","v_CA16_2510","v_CA16_2525","v_CA16_5096","v_CA16_5147"), labels="detailed", geo_format='sf', level='DA')
# filter('`Region Name`' == 'Vancouver') # Installing Census data api but failed to select only Vancouver region

Reading vectors data from local cache.

Reading geo data from local cache.



In [45]:
colnames(census_data)

[1] "Shape Area"                                                                                                                                    
 [2] "Type"                                                                                                                                          
 [3] "Households"                                                                                                                                    
 [4] "CD_UID"                                                                                                                                        
 [5] "Dwellings"                                                                                                                                     
 [6] "GeoUID"                                                                                                                                        
 [7] "Population"                                                                                                                                    
 [8] "CSD_UID"                                                                                                                                       
 [9] "CT_UID"                                                                                                                                        
[10] "CMA_UID"                                                                                                                                       
[11] "Region Name"                                                                                                                                   
[12] "Area (sq km)"                                                                                                                                  
[13] "v_CA16_401: Population, 2016"                                                                                                                  
[14] "v_CA16_404: Total private dwellings"                                                                                                           
[15] "v_CA16_1: Age Stats"                                                                                                                           
[16] "v_CA16_2426: Total - Household after-tax income groups in 2015 for private households - 100% data"                                             
[17] "v_CA16_2510: Total - Low-income status in 2015 for the population in private households to whom low-income concepts are applicable - 100% data"
[18] "v_CA16_2525: In low income based on the Low-income measure, after tax (LIM-AT)"                                                                
[19] "v_CA16_5096: Total - Highest certificate, diploma or degree for the population aged 25 to 64 years in private households - 25% sample data"    
[20] "v_CA16_5147: Education"                                                                                                                        
[21] "geometry"

In [46]:
glimpse(census_data)

Rows: 10
Columns: 21
$ `Shape Area`                                                                                                                                     <dbl> …
$ Type                                                                                                                                             <fct> …
$ Households                                                                                                                                       <int> …
$ CD_UID                                                                                                                                           <chr> …
$ Dwellings                                                                                                                                        <int> …
$ GeoUID                                                                                                                                           <chr> …
$ Population                                     

In [47]:
install.packages("sf")
library("sf")

Warning message:
"package 'sf' is in use and will not be installed"


In [48]:
library(VancouvR)
search_cov_datasets("property-tax") %>%
  select(dataset_id,title)

dataset_id,title
<chr>,<chr>
property-tax-report-2011-2015,Property tax report 2011-2015
property-tax-report-2006-2010,Property tax report 2006-2010
property-tax-report-2016-2019,Property tax report 2016-2019
property-tax-report,Property tax report


In [49]:
tax_data <- get_cov_data(dataset_id = "property-tax-report-2016-2019",
                         where="tax_assessment_year='2016'",
                         select = "current_land_value, land_coordinate as tax_coord")|>
    as.tibble()
glimpse(tax_data) # read in the data of land value, and changed it coordname to tax coord, so it matched the property polygon data

Reading data from temporary cache

Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `current_land_value =
  .Primitive("as.integer")(current_land_value)`.
Caused by warning:
! NAs introduced by coercion to integer range"


Rows: 199,924
Columns: 2
$ current_land_value <int> 362000, 4217000, 346000, 287000, 1883000, 228000, 1…
$ tax_coord          <chr> "12060598", "58911893", "60318606", "64615992", "69…


In [50]:
colnames(tax_data)

[1] "current_land_value" "tax_coord"

In [51]:
url = "https://raw.githubusercontent.com/Irene-Berezin/econ326/refs/heads/main/crimedata_csv_AllNeighbourhoods_AllYears/crimedata_csv_AllNeighbourhoods_AllYears.csv"
intentional_crime_data <- read_csv(url)|>
    as.tibble()|>
    select(TYPE, YEAR, NEIGHBOURHOOD)|>
    rename(type = TYPE, year = YEAR, neighbourhood = NEIGHBOURHOOD)|>
    filter(!type %in% c("Vehicle Collision or Pedestrian Struck (with Fatality)",
    "Vehicle Collision or Pedestrian Struck (with Injury)",
    "Mischief"))|>
    filter(year == 2016)|>
    group_by(type)|>
    arrange(year)

glimpse(intentional_crime_data)


Rows: 895823 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (3): TYPE, HUNDRED_BLOCK, NEIGHBOURHOOD
dbl (7): YEAR, MONTH, DAY, HOUR, MINUTE, X, Y

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Rows: 37,802
Columns: 3
Groups: type [8]
$ type          <chr> "Offence Against a Person", "Offence Against a Person", …
$ year          <dbl> 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 2016, 20…
$ neighbourhood <chr> "Kitsilano", "Kitsilano", "Kitsilano", "Kitsilano", "Kit…


In [52]:
local_area_boundary <- get_cov_data(dataset_id ="local-area-boundary")|>
    glimpse()

Reading data from temporary cache



Rows: 22
Columns: 4
$ name         <chr> "Arbutus Ridge", "Grandview-Woodland", "Kensington-Cedar …
$ geom         <chr> "{\"coordinates\": [[[-123.15259552001952, 49.25722503568…
$ geo_point_2d <chr> "49.24680490982789, -123.16166923820482", "49.27643960930…
$ geometry     <POLYGON [°]> POLYGON ((-123.1526 49.2572..., POLYGON ((-123.07…


In [53]:
glimpse(local_area_boundary)  # This dataframe stores the geom

Rows: 22
Columns: 4
$ name         <chr> "Arbutus Ridge", "Grandview-Woodland", "Kensington-Cedar …
$ geom         <chr> "{\"coordinates\": [[[-123.15259552001952, 49.25722503568…
$ geo_point_2d <chr> "49.24680490982789, -123.16166923820482", "49.27643960930…
$ geometry     <POLYGON [°]> POLYGON ((-123.1526 49.2572..., POLYGON ((-123.07…


In [54]:
property_polygons <- get_cov_data(dataset_id="property-parcel-polygons")

Reading data from temporary cache



In [55]:
glimpse(property_polygons)

Rows: 99,939
Columns: 7
$ civic_number <chr> "4050", "4034", "4026", "4033", "4041", "4055", "5912", "…
$ streetname   <chr> "W 40TH AV", "W 40TH AV", "W 40TH AV", "W 41ST AV", "W 41…
$ tax_coord    <chr> "74703048", "74703062", "74703070", "75003063", "75003057…
$ site_id      <chr> "011037491", "009828028", "011037521", "011037971", "0110…
$ geom         <chr> "{\"coordinates\": [[[-123.1952054199714, 49.235263660093…
$ geo_point_2d <chr> "49.23541759077318, -123.19531316487794", "49.23541632098…
$ geometry     <POLYGON [°]> POLYGON ((-123.1952 49.2352..., POLYGON ((-123.19…


In [56]:
census_geom <- st_geometry(census_data)
census_centroids <- st_centroid(census_geom)

In [ ]:
land_price <- merge(tax_data, property_polygons, by = "tax_coord")|>
    glimpse() 

# Merge two datasets so we have land price and their geometry information in one single dataframe

Rows: 204,278
Columns: 8
$ tax_coord          <chr> "01963206", "01963229", "01963245", "01963265", "01…
$ current_land_value <int> 11366000, 10623000, 10763000, 10763000, 12864000, 1…
$ civic_number       <chr> "4888", "4898", "4874", "4850", "4811", "4818", "16…
$ streetname         <chr> "BELMONT AV", "FANNIN AV", "FANNIN AV", "FANNIN AV"…
$ site_id            <chr> "010862706", "010863915", "010863923", "010863931",…
$ geom               <chr> "{\"coordinates\": [[[-123.22221178283283, 49.27466…
$ geo_point_2d       <chr> "49.27488185499064, -123.22286857659084", "49.27431…
$ geometry           <POLYGON [°]> POLYGON ((-123.2222 49.2746..., POLYGON ((-…
